In [45]:
sentences = [
    "The sky turned a fiery orange as the sun dipped below the horizon, casting long shadows across the tranquil meadow.",
    "Lost in thought, she wandered through the labyrinth of bustling streets, each alleyway revealing a new adventure waiting to unfold.",
    "With a flicker of hesitation, he reached out and grasped the ancient artifact, feeling its power surge through his veins.",
    "Laughter echoed through the halls as friends gathered around the crackling fireplace, swapping stories late into the night.",
    "Time seemed to stand still as they danced beneath the twinkling stars, their hearts beating in perfect harmony.",
    "The aroma of freshly brewed coffee filled the air, awakening her senses with each comforting sip.",
    "Thunder rumbled ominously in the distance, signaling an approaching storm that would soon unleash its fury upon the unsuspecting town.",
    "In the quiet solitude of the forest, she found solace among the towering trees, their gentle whispers soothing her troubled mind.",
    "With a flourish of his pen, he signed his name at the bottom of the contract, sealing the deal with a sense of satisfaction.",
    "As dawn broke over the horizon, painting the sky in hues of pink and gold, she knew that today held endless possibilities waiting to be discovered."
]

In [46]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import tqdm as notebook_tqdm

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [49]:
tokenized_sentences = tokenizer(sentences, return_tensors = 'pt', max_length = 32, padding = 'max_length', truncation = True)
tokenized_sentences['labels'] = tokenized_sentences.input_ids.detach().clone()

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors='pt')
masked = data_collator.torch_mask_tokens(tokenized_sentences.input_ids)

data_dict = {}
for i in masked:
    data_dict.update({'input_ids':masked[0][i], 'attention_mask':masked[1][i], 'labels':tokenized_sentences.input_ids[i]})
data = {'input_ids':masked[0], 'attention_mask':masked[1], 'labels':tokenized_sentences.input_ids}

In [51]:
tokenized_sentences.input_ids[1]

tensor([  101,  2439,  1999,  2245,  1010,  2016, 13289,  2083,  1996, 24239,
         1997, 13950,  2989,  4534,  1010,  2169,  8975,  4576,  8669,  1037,
         2047,  6172,  3403,  2000,  4895, 10371,  1012,   102,     0,     0,
            0,     0])

In [42]:
masked[0]

tensor([[  101,  1996,  3712,   103,  1037, 15443,  4589,   103,  1996, 11690,
           103,   103,  1996,  9154,  1010,   103,  2146,  6281,  2408, 21391,
         25283,   103,   103,  1012,   102,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  2439,   103,  2245,  1010,  2016, 13289,  2083,  1996, 24239,
         12968,   103,  2989,   103,  1010,   103,  8975,  4576,  8669,  1037,
          2047,   103,  3403,  2000,  4895,   103,   103,   102,     0,     0,
             0,     0],
        [  101,  2007,  1037, 17909,  1997,   103,  1010,   103,  2584,  2041,
           103, 15517,  1996,  3418, 20785,   103,   103,  2049,   341, 16859,
          2083,  2010,  9607,  1012,   102,     0,     0,     0,     0,     0,
             0,     0],
        [  101,   103, 10187,  2083,  1996,  9873,  2004,  2814,  5935,   103,
          1996,  8579,   103, 13788,  1010, 19948,  4691,  3441,  2397,  2046,
           103,  2305,  1012,   102,     0,     0,     0,  

In [27]:
import torch

dataloader = torch.utils.data.DataLoader(data, batch_size=1)

In [8]:
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

In [9]:
from torch.optim import AdamW
from transformers import get_scheduler

optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 5
num_training_steps = num_epochs * len(sentences)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

In [10]:
device

device(type='cuda')

In [10]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    print(epoch)
    for sentence in dataloader:
        data = {k: v.to(device) for k, v in sentence.items()}
        outputs = model(**data)
        loss = outputs.loss
        print(loss)
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  2%|▏         | 1/50 [00:54<44:07, 54.03s/it]

0


KeyError: 0

In [33]:
for batch in dataloader:
    print(batch)

KeyError: 0

In [28]:
sentence['input_ids'][0][2]

tensor([  101,  2007,  1037, 17909,  1997, 13431,  1010,   103,  2584,  2041,
          103, 15517,  1996,  3418, 20785,  1010,  3110,  2049,  2373, 12058,
         2083,  2010,  9607,  1012,   102,     0,     0,     0,     0,     0,
            0,     0])